In [1]:
# Import specific functions
import sys
import numpy as np
from numpy.linalg import eig, inv, pinv, eigvals
from scipy.spatial.distance import squareform as squareform_sp
from scipy.sparse import csr_matrix, random as sparse_random, find, issparse
# from scipy.stats import multivariate_normal as mvnpdf
from pynndescent import NNDescent
import time
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import inv
import numpy as np
import scipy.sparse as sp
from scipy.spatial.distance import squareform
from scipy.sparse import triu, coo_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, normalized_mutual_info_score
from scipy.sparse.linalg import norm as sparse_norm
from scipy.spatial.distance import squareform
from math import sqrt
import networkx as nx
import matplotlib.pyplot as plt
from scipy.linalg import cholesky


sys.path.append('/Users/paul_reitz/Documents/repos/PyAWGLMM/Smooth_AWGLMM')
from scripts.utils import (
    visualize_glmm,
    graph_learning_perf_eval,
    identify_and_compare,
    generate_connected_graph,
    normest,
    lin_map,
    # squareform_sp,
    sum_squareform,
    prox_sum_log,
    gsp_distanz,
    gsp_compute_graph_learning_theta,
    gsp_symmetrize,
)
from scripts.utils_deep import mvnpdf

np.random.seed(17307946)

In [2]:
def sum_squareform(n, mask=None):
    """
    Python version of the MATLAB function sum_squareform.
    
    Parameters:
    -----------
    n : int
        Size of the matrix W (an n-by-n matrix).
    mask : array-like or None
        If given, must be of size n(n-1)/2. Non-zero entries indicate
        which elements in w = squareform(W) are considered.
    
    Returns:
    --------
    S : scipy.sparse.csc_matrix
        Matrix so that S * w = sum(W) for w = squareform(W).
    St : scipy.sparse.csc_matrix
        The adjoint of S (S transpose).
        
    The output is consistent with the MATLAB function:
    [S, St] = sum_squareform(n, mask)
    """
    if mask is None:
        mask_given = False
    else:
        mask_given = True

    if mask_given:
        # Check mask length
        if len(mask) != n*(n-1)//2:
            raise ValueError('mask size must be n(n-1)/2')
        
        # Find nonzero indices in mask
        # MATLAB code tries to handle both column vectors and row vectors
        # In Python, let's just do a generic approach:
        mask = np.asarray(mask).flatten()
        ind_vec = np.nonzero(mask)[0]  # 0-based indices of nonzero
        
        ncols = len(ind_vec)

        I = np.zeros(ncols, dtype=np.int64)
        J = np.zeros(ncols, dtype=np.int64)

        curr_row = 1
        offset = 0
        # length of current row of matrix, counting from after the diagonal
        length = n - 1
        for ii in range(ncols):
            ind_vec_i = ind_vec[ii] + 1  # switch to 1-based for logic
            # Move down rows until we find the correct row
            while ind_vec_i > (length + offset):
                offset += length
                length -= 1
                curr_row += 1
            # curr_row, J position in MATLAB indexing
            I[ii] = curr_row
            # Convert the position within the row into an absolute column index
            # J(ii) = ind_vec_i - offset + (n - length)
            J[ii] = ind_vec_i - offset + (n - length)

    else:
        # No mask given
        ncols = (n-1)*n//2
        I = np.zeros(ncols, dtype=np.int64)
        J = np.zeros(ncols, dtype=np.int64)

        # Fill I
        k = 0
        for i in range(2, n+1):
            end_idx = k+(n - i + 1)
            # I(k : k+(n-i)) = i:n in MATLAB (1-based)
            # That means for each i, we fill a block of size (n-i+1) with values from i to n.
            I[k:end_idx] = np.arange(i, n+1)
            k = end_idx

        # Fill J
        k = 0
        for i in range(2, n+1):
            end_idx = k+(n - i + 1)
            # J(k : k+(n-i)) = i-1 in MATLAB
            # That means for each block, we fill it with i-1 repeated.
            J[k:end_idx] = (i - 1)
            k = end_idx

    # Convert from 1-based to 0-based indexing
    I -= 1
    J -= 1

    # Construct St
    # St in MATLAB: sparse([1:ncols, 1:ncols],[I, J],1,ncols,n)
    # means each row i has two ones: one at column I(i), one at column J(i).
    # In Python (0-based), rows = 0..ncols-1
    row_indices = np.repeat(np.arange(ncols), 2)
    col_indices = np.hstack((I, J))
    data = np.ones(2*ncols, dtype=np.float64)

    St = coo_matrix((data, (row_indices, col_indices)), shape=(ncols, n)).tocsc()
    S = St.transpose().tocsc()

    return S, St


In [3]:
# def gsp_distanz_pynndescent(X, Y=None, P=None, n_neighbors=10, metric='euclidean'):
#     """
#     gsp_distanz_pynndescent calculates the approximate distances between vectors in X and Y
#     using the pynndescent library for nearest neighbors.

#     Parameters
#     ----------
#     X : ndarray
#         Matrix with column vectors (shape: n x d, where n is the number of vectors and d is the dimension).
#     Y : ndarray, optional
#         Matrix with column vectors (shape: m x d). Default is X.
#     P : ndarray, optional
#         Not supported in this implementation. Raises NotImplementedError if provided.
#     n_neighbors : int, optional
#         Number of nearest neighbors to compute. Default is 10.
#     metric : str or callable, optional
#         Distance metric to use. Default is 'euclidean'. Must be supported by pynndescent.

#     Returns
#     -------
#     D : csr_matrix
#         Sparse distance matrix of shape (n x m) where each row contains distances to the nearest neighbors.
    
#     Indices : csr_matrix
#         Sparse matrix containing the indices of the nearest neighbors.
    
#     Notes
#     -----
#     - This function uses approximate nearest neighbors for efficiency.
#     - If Y is provided and different from X, it computes neighbors from X to Y.
#     - If Y is None, it computes neighbors within X.
#     - The output distance matrix is sparse, containing distances only to the nearest neighbors.
#     """
#     if X is None:
#         raise ValueError("Input matrix X must be provided.")
    
#     if P is not None:
#         raise NotImplementedError("Custom metric matrices (P) are not supported in this implementation.")
    
#     if Y is None:
#         Y = X

#     if X.shape[1] != Y.shape[1]:
#         raise ValueError("The dimensionality of X and Y must match.")
    
#     # Initialize NNDescent with X as the dataset to search
#     # If Y is provided and different, use X as query and Y as the dataset
#     if np.array_equal(X, Y):
#         # Search within X
#         nnd = NNDescent(X, n_neighbors=n_neighbors, metric=metric)
#         neighbors, distances = nnd.query(X, k=n_neighbors)
#     else:
#         # Search from X to Y
#         nnd = NNDescent(Y, n_neighbors=n_neighbors, metric=metric)
#         neighbors, distances = nnd.query(X, k=n_neighbors)
    
#     # Construct a sparse distance matrix
#     # Rows correspond to each vector in X, columns to their k neighbors in Y
#     n_samples_X = X.shape[0]
#     n_samples_Y = Y.shape[0]
    
#     # Flatten the data for constructing sparse matrix
#     row_indices = np.repeat(np.arange(n_samples_X), n_neighbors)
#     col_indices = neighbors.flatten()
#     data = distances.flatten()
    
#     # Create a sparse matrix in CSR format
#     D = csr_matrix((data, (row_indices, col_indices)), shape=(n_samples_X, n_samples_Y))
    
#     return D , distances

In [4]:

# def gsp_distanz_ann(X, Y=None, P=None):
#     """
#     Computes approximate distances between all column vectors in X and Y using PyNNDescent.
#     If P is provided, performs a linear transformation to emulate the Mahalanobis-like distance.

#     Parameters
#     ----------
#     X : ndarray of shape (d, n)
#         Data matrix with column vectors.
#     Y : ndarray of shape (d, m), optional
#         Data matrix with column vectors. If not provided, Y = X.
#     P : ndarray of shape (d, d), optional
#         Positive semidefinite matrix for generalized distance:
#         D_ij = sqrt((X_i - Y_j)^T P (X_i - Y_j))
#         If not provided, identity (Euclidean) is assumed.
        
#     Returns
#     -------
#     D : ndarray of shape (n, m)
#         Approximate distance matrix, where D[i,j] = distance between X_i and Y_j.
#     """
#     if X is None:
#         raise ValueError("Not enough input parameters: X must be provided")

#     if Y is None:
#         Y = X

#     rx, cx = X.shape
#     ry, cy = Y.shape
#     if rx != ry:
#         raise ValueError("The dimensions of X and Y must match along rows!")

#     # If P is provided, factor it to transform the data
#     if P is not None:
#         rp, rp2 = P.shape
#         if rx != rp or rp != rp2:
#             raise ValueError("P must be square and match the dimension of X")

#         # Cholesky decomposition (or any factorization) to get L: P = L^T L
#         # For numerical stability and correctness, ensure P is PSD.
#         L = cholesky(P, lower=True)  # L lower triangular, so P = L.T @ L

#         # Transform X and Y: X' = L*X and Y' = L*Y
#         X_trans = L @ X
#         Y_trans = L @ Y
#     else:
#         # If no P, we just use X and Y directly
#         X_trans = X
#         Y_trans = Y

#     # PyNNDescent expects (n_samples, n_features)
#     data = X_trans.T  # shape: (n, d)
#     query = Y_trans.T # shape: (m, d)
#     n = data.shape[0]
#     m = query.shape[0]

#     # Build the ANN index
#     index = NNDescent(data, metric='euclidean')
#     # Query all points to reconstruct full distance matrix
#     # This can be very expensive for large n!
#     indices, distances = index.query(query, k=n)

#     # Reconstruct the full distance matrix
#     D = np.zeros((m, n), dtype=float)
#     for i in range(m):
#         # indices[i] are the indices of the neighbors for query i
#         # distances[i] are the corresponding approximate distances
#         D[i, indices[i]] = distances[i]

#     # If Y is X, zero out the diagonal
#     if Y is X:
#         np.fill_diagonal(D, 0.0)

#     # Return D in shape (n, m), currently it's (m, n), so we transpose
#     D = D.T

#     # Add a small epsilon and sqrt is not needed since distances from NNDescent
#     # are already Euclidean distances. Still, we can clamp small negatives:
#     eps = 1e-14
#     D = np.maximum(D, eps)
#     # No need to sqrt here because PyNNDescent returns distances directly.
#     # But if we wanted to ensure distances are always sqrt-distances, they already are.

#     return D

In [5]:
def gsp_ann_distanz(X, Y=None, k=10, metric_in = 'euclidean'):
    """
    Compute an approximate k-NN distance matrix using PyNNDescent.
    
    Parameters
    ----------
    X : ndarray of shape (d, n)
        Data matrix with column vectors.
    Y : ndarray of shape (d, m), optional
        Query matrix with column vectors. If not provided, Y = X.
    k : int
        Number of nearest neighbors to compute for each vector.
        
    Returns
    -------
    Z : ndarray of shape (n, m)
        Approximate k-NN distance matrix. For each column in Y, we store 
        the distances to the k-nearest neighbors in X in the corresponding 
        rows, and 0 in other positions. If Y is X, Z will be symmetric 
        except for the zeroing of non-neighbors.
        
    Notes
    -----
    - Z[i,j] = distance between X_i and Y_j if X_i is among the k-nearest 
      neighbors of Y_j.
    - If Y is X, we remove self-distances by setting the diagonal to 0.0.
    - Non-neighbor distances are set to 0. If you prefer another placeholder,
      you could use np.inf instead.
    """
    if X is None:
        raise ValueError("X must be provided")

    if Y is None:
        Y = X

    dX, n = X.shape
    dY, m = Y.shape

    if dX != dY:
        raise ValueError("X and Y must have the same number of rows (dimensions)")

    # Build the ANN index from X
    data = X.T  # shape: (n, d)
    query = Y.T # shape: (m, d)
    
    # Use PyNNDescent to build and query
    index = NNDescent(data, metric=metric_in, n_trees=10, n_iters=10)
    # Query for the k nearest neighbors
    indices, distances = index.query(query, k=k)

    # Now we build a (n x m) matrix Z
    # Initialize with zeros
    Z = np.zeros((n, m))
    # For each column vector in Y, place its k-nearest neighbor distances
    # in the corresponding positions.
    # indices[i] gives the k-nearest neighbors of Y_i in X
    # distances[i] gives the corresponding distances
    for i in range(m):
        Z[indices[i], i] = distances[i]

    # If Y is X, ensure diagonal is zero (distance to self)
    if Y is X:
        np.fill_diagonal(Z, 0.0)

    return Z

In [6]:
def normalize_data(y):
    # Standardize each feature to zero mean, unit variance
    mean = np.mean(y, axis=0)
    std = np.std(y, axis=0) + 1e-12
    y_norm = (y - mean)/std
    return y_norm


In [7]:
def generate_connected_graph(n, p, zero_thresh, maxit=10, verbose=1):
    """
    Generate a connected Erdos-Renyi graph using networkx and ensure 
    its second smallest eigenvalue of Laplacian is > zero_thresh.

    Parameters
    ----------
    n : int
        Number of nodes.
    p : float
        Probability of connection between nodes.
    zero_thresh : float
        Threshold for the second smallest eigenvalue to ensure connectivity.
    maxit : int, optional
        Maximum number of tries to get a connected graph (default 10).
    verbose : int, optional
        Verbosity level (default 1).

    Returns
    -------
    G_dict : dict
        A dictionary mimicking the G structure with keys:
        - 'W': adjacency/weight matrix
        - 'L': Laplacian matrix
        - 'N': number of nodes
        - 'type': 'erdos_renyi'
    
    Raises
    ------
    ValueError
        If after maxit attempts no connected graph is found.
    """
    for iteration in range(1, maxit+1):
        # Generate an Erdos-Renyi graph
        Gnx = nx.erdos_renyi_graph(n, p)
        W = nx.to_numpy_array(Gnx, dtype=np.float64)

        # Remove any self loops by zeroing diagonal
        np.fill_diagonal(W, 0.0)

        # Compute Laplacian
        d = np.sum(W, axis=1)
        D = np.diag(d)
        L = D - W

        # Compute eigenvalues
        e = np.linalg.eigvalsh((L + L.T)*0.5)
        e = np.sort(e)

        # Check connectivity condition
        if len(e) > 1 and e[1] > zero_thresh:
            if verbose > 1:
                print(f"A connected graph has been created in {iteration} iteration(s)")

            G_dict = {}
            G_dict['W'] = W
            G_dict['L'] = L
            G_dict['N'] = n
            G_dict['type'] = 'erdos_renyi'
            return G_dict
        else:
            if verbose > 1:
                print(f"Iteration {iteration} failed. Trying again.")

    if verbose:
        print("Warning: The graph is not strongly connected after maxit attempts.")
    raise ValueError("Could not generate a connected graph after maxit attempts.")







In [15]:
import numpy as np

def gsp_learn_graph_log_degrees(Z, a, b, params=None):
    """
    Python version of gsp_learn_graph_log_degrees.

    Parameters
    ----------
    Z : array-like
        Matrix or vector of (squared) pairwise distances.
    a : float
        Log prior constant.
    b : float
        ||W||_F^2 prior constant.
    params : dict, optional
        Dictionary of parameters:
            - verbosity : int, default=1
            - maxit : int, default=1000
            - tol : float, default=1e-5
            - step_size : float in (0,1), default=0.5
            - fix_zeros : bool, default=False
            - max_w : float, default=inf
            - w_0 : array-like or scalar, optional
            - c : float, required if w_0 given
            - edge_mask : array-like, required if fix_zeros=True

    Returns
    -------
    W : ndarray
        Learned weighted adjacency matrix (or vector if Z was vector).
    stat : dict
        Dictionary with stats about the optimization.
    """

    if params is None:
        params = {}
    verbosity = params.get('verbosity', 1)
    maxit = params.get('maxit', 1000)
    tol = params.get('tol', 1e-5)
    step_size = params.get('step_size', 0.5)
    fix_zeros = params.get('fix_zeros', False)
    max_w = params.get('max_w', np.inf)

    # Convert Z to vector form if needed
    # Assume we have squareform_sp implemented
    if Z.ndim > 1:
        z = squareform_sp(Z)
    else:
        z = Z
    z = z.flatten()

    l = len(z)  # number of edges
    # n(n-1)/2 = l => n = (1 + sqrt(1+8*l))/2
    n = int(round((1 + np.sqrt(1+8*l))/2))

    if 'w_0' in params:
        w_0 = params['w_0']
        if 'c' not in params:
            raise ValueError("When params.w_0 is specified, params.c must also be specified")
        c = params['c']
        if w_0.ndim > 1:
            w_0 = squareform_sp(w_0)
        w_0 = w_0.flatten()
    else:
        w_0 = 0
        c = 0  # not used if w_0=0

    # If fix_zeros is set, we learn only a subset of edges
    if fix_zeros:
        edge_mask = params['edge_mask']
        if edge_mask.ndim > 1:
            edge_mask = squareform_sp(edge_mask)
        edge_mask = edge_mask.flatten()
        ind = np.nonzero(edge_mask)[0]
        z = z[ind]
        if not np.isscalar(w_0):
            w_0 = w_0[ind]
    else:
        # Make sure arrays are full
        z = np.array(z, dtype=float)
        if not np.isscalar(w_0):
            w_0 = np.array(w_0, dtype=float)

    # Initialization
    # If W_init given, use it
    if 'W_init' in params:
        w = params['W_init'].copy()
        if w.ndim > 1:
            w = squareform_sp(w)
        w = w.flatten()
        if fix_zeros:
            w = w[ind]
    else:
        w = np.zeros_like(z)

    # Get S, St from sum_squareform
    if fix_zeros:
        S, St = sum_squareform(n, edge_mask)
    else:
        S, St = sum_squareform(n)

    # Define linear operators K and Kt
    K_op = lambda w: S @ w
    Kt_op = lambda z: St @ z

    # norm_K (used for step sizing)
    if fix_zeros:
        norm_K = normest(S)
    else:
        norm_K = np.sqrt(2*(n-1))

    # Define f, g, h as in the code
    # f(w) = 2*w'*z, prox_f(w,c) = min(max_w, max(0, w - 2*c*z))
    def f_eval(w_):
        return 2 * np.dot(w_, z)

    def f_prox(w_, c_):
        return np.minimum(max_w, np.maximum(0, w_ - 2*c_*z))

    # g(w) = -a * sum(log(sum(W)))
    # We handle g through g_star_prox. We have prox_sum_log already.
    def g_eval_edges(w_):
        # g(K_op(w)) = -a * sum(log(K_op(w))) 
        tmp = K_op(w_)
        return -a * np.sum(np.log(tmp))

    param_prox_log = {'verbose': verbosity - 3}
    # def g_star_prox(z_, c_):
    #     # z - c*a * prox_sum_log(z/(c*a), 1/(c*a))
    #     return z_ - c_*a * prox_sum_log(z_/(c_*a), 1/(c_*a), param_prox_log)
    
    def g_star_prox(z_, c_):
        prox_val, _ = prox_sum_log(z_ / (c_ * a), 1 / (c_ * a), param_prox_log)  # Unpack the tuple
        prox_val = np.array(prox_val, dtype=float) 
        return z_ - c_*a * prox_val

    # h(w) = b||w||_F^2 + c||w - w_0||_F^2 if w_0 given
    if np.isscalar(w_0):
        def h_eval(w_):
            return b * np.sum(w_**2)
        def h_grad(w_):
            return 2 * b * w_
        h_beta = 2 * b
    else:
        def h_eval(w_):
            return b * np.sum(w_**2) + c * np.sum((w_ - w_0)**2)
        def h_grad(w_):
            return 2 * ((b+c)*w_ - c*w_0)
        h_beta = 2 * (b + c)

    # Parameters for FBF
    mu = h_beta + norm_K
    epsilon = lin_map(0.0, [0, 1/(1+mu)], [0,1])
    gn = lin_map(step_size, [epsilon, (1 - epsilon)/mu], [0,1])

    # Dual variable
    v_n = K_op(w)

    # statistics
    stat = {}
    if verbosity > 1 or True:
        stat['f_eval'] = np.full((maxit,), np.nan)
        stat['g_eval'] = np.full((maxit,), np.nan)
        stat['h_eval'] = np.full((maxit,), np.nan)
        stat['fgh_eval'] = np.full((maxit,), np.nan)
        stat['pos_violation'] = np.full((maxit,), np.nan)

    if verbosity > 1:
        print('Relative change of primal, dual variables, and objective fun')

    start_time = 0.0
    import time
    start_time = time.time()

    for i in range(maxit):
        # Forward-backward-forward updates
        Y_n = w - gn * (h_grad(w) + Kt_op(v_n))
        y_n = v_n + gn * (K_op(w))
        P_n = f_prox(Y_n, gn)
        p_n = g_star_prox(y_n, gn)
        Q_n = P_n - gn * (h_grad(P_n) + Kt_op(p_n))
        q_n = p_n + gn * (K_op(P_n))

        # Evaluate objective and constraints if needed
        stat['f_eval'][i] = f_eval(w)
        stat['g_eval'][i] = g_eval_edges(w)
        stat['h_eval'][i] = h_eval(w)
        stat['fgh_eval'][i] = stat['f_eval'][i] + stat['g_eval'][i] + stat['h_eval'][i]
        stat['pos_violation'][i] = -np.sum(np.minimum(0, w))

        # stopping criterion
        rel_norm_primal = np.linalg.norm(-Y_n + Q_n) / (np.linalg.norm(w) + 1e-15)
        rel_norm_dual = np.linalg.norm(-y_n + q_n) / (np.linalg.norm(v_n) + 1e-15)

        if verbosity > 2:
            print(f'iter {i+1:4d}: {rel_norm_primal:6.4e}   {rel_norm_dual:6.4e}   {stat["fgh_eval"][i]:6.3e}')
        elif verbosity > 1:
            print(f'iter {i+1:4d}: {rel_norm_primal:6.4e}   {rel_norm_dual:6.4e}')

        w = w - Y_n + Q_n
        v_n = v_n - y_n + q_n

        if rel_norm_primal < tol and rel_norm_dual < tol:
            break

    stat['time'] = time.time() - start_time
    if verbosity > 0:
        print(f'# iters: {i+1:4d}. Rel primal: {rel_norm_primal:6.4e} Rel dual: {rel_norm_dual:6.4e}  OBJ {stat["fgh_eval"][i]:6.3e}')
        print(f'Time needed is {stat["time"]:.6f} seconds')

    # If fix_zeros, we must put values back in their original positions
    if fix_zeros:
        full_w = np.zeros(l)
        full_w[ind] = w
        w = full_w

    # If input was a matrix, return W as a matrix
    if Z.ndim > 1:
        W = squareform_sp(w)
    else:
        W = w

    return W, stat


#########################################
# Tests to verify correctness
#########################################
if __name__ == "__main__":
    # Simple test: small random graph with distances
    np.random.seed(0)
    n = 5
    X = np.random.randn(n, 3)  # some signals
    # Suppose Z is the squared distance matrix of X
    # If you have gsp_distanz implemented:
    # Z = gsp_distanz(X.T)**2
    # For now, let's just create a random symmetric matrix with zeros on diag:
    D = np.random.rand(n, n)
    Z = (D + D.T)/2
    np.fill_diagonal(Z, 0)
    # Ensure positive distances
    Z = np.abs(Z)

    # Parameters
    a = 1.0
    b = 1.0
    params = {
        'verbosity': 2,
        'maxit': 50,
        'tol': 1e-4,
        'step_size': 0.5,
        'fix_zeros': False,
        'max_w': np.inf
    }

    W, stat = gsp_learn_graph_log_degrees(Z, a, b, params)

    # Check W is symmetric and no negative edges:
    assert np.allclose(W, W.T), "W should be symmetric"
    assert np.all(W >= -1e-12), "W should be non-negative"
    # Check dimensions
    assert W.shape == (n, n), "W should be n-by-n for matrix input"
    print("Basic test passed!")

    # Test with vector input (just a single edge vector)
    n = 4
    Z_vec = np.random.rand(n*(n-1)//2)
    W_vec, stat_vec = gsp_learn_graph_log_degrees(Z_vec, a, b, params)
    # Should return vector form since input was vector
    assert W_vec.shape == Z_vec.shape, "Output should have same shape as input vector"
    print("Vector input test passed!")


Relative change of primal, dual variables, and objective fun
iter    1: 2.1075e+14   7.1956e+14
iter    2: 2.9873e-01   6.1221e-01
iter    3: 4.1284e-01   2.6114e-01
iter    4: 3.7776e-01   1.4753e-01
iter    5: 2.9834e-01   8.9720e-02
iter    6: 2.2558e-01   5.3847e-02
iter    7: 1.6798e-01   3.0145e-02
iter    8: 1.2352e-01   1.4864e-02
iter    9: 8.9358e-02   8.2446e-03
iter   10: 6.3193e-02   9.9872e-03
iter   11: 4.3327e-02   1.2621e-02
iter   12: 2.8542e-02   1.3735e-02
iter   13: 1.7995e-02   1.3413e-02
iter   14: 1.1179e-02   1.2084e-02
iter   15: 7.8062e-03   1.0184e-02
iter   16: 6.9856e-03   8.0759e-03
iter   17: 7.0370e-03   6.0270e-03
iter   18: 6.9516e-03   4.2112e-03
iter   19: 6.5069e-03   2.7254e-03
iter   20: 5.7704e-03   1.6185e-03
iter   21: 4.8733e-03   9.3680e-04
iter   22: 3.9380e-03   7.2705e-04
iter   23: 3.0579e-03   7.8951e-04
iter   24: 2.2952e-03   8.5604e-04
iter   25: 1.6850e-03   8.5458e-04
iter   26: 1.2406e-03   7.9230e-04
iter   27: 9.5368e-04   6.911

/var/folders/0t/4q2l76sj2v91nwcfqht0bwlw0000gn/T/ipykernel_60453/4444524.py:125: RuntimeWarning: divide by zero encountered in log
  return -a * np.sum(np.log(tmp))


In [10]:

# def gsp_learn_graph_log_degrees(Z, a, b, params=None):
#     if params is None:
#         params = {}
#     verbosity = params.get('verbosity', 1)
#     maxit = params.get('maxit', 1000)
#     tol = params.get('tol', 1e-5)
#     step_size = params.get('step_size', 0.5)
#     fix_zeros = params.get('fix_zeros', False)
#     max_w = params.get('max_w', np.inf)

#     w_0 = params.get('w_0', 0)
#     if w_0 != 0 and 'c' not in params:
#         raise ValueError("When w_0 is specified, c must also be specified")
#     c = params.get('c', 0.0 if w_0 == 0 else None)

#     # Convert Z to vector form
#     if Z.ndim == 2 and Z.shape[0] == Z.shape[1]:
#         z = squareform_sp(Z)
#     else:
#         z = Z
#     z = z.ravel()
#     l = len(z)
#     n = int(round((1 + sqrt(1+8*l))/2))

#     if not np.isscalar(w_0):
#         # Convert w_0 to vector form if needed
#         if w_0.ndim == 2 and w_0.shape[0] == w_0.shape[1]:
#             w_0 = squareform_sp(w_0)
#         w_0 = w_0.ravel()
#     else:
#         w_0 = float(w_0)

#     if fix_zeros:
#         edge_mask = params.get('edge_mask', None)
#         if edge_mask is None:
#             raise ValueError("edge_mask must be provided when fix_zeros is True")
#         if edge_mask.ndim == 2 and edge_mask.shape[0] == edge_mask.shape[1]:
#             edge_mask = squareform_sp(edge_mask)
#         edge_mask = edge_mask.ravel()
#         ind = np.flatnonzero(edge_mask)
#         z = z[ind].astype(float)
#         if not np.isscalar(w_0):
#             w_0 = w_0[ind].astype(float)
#     else:
#         z = z.astype(float)
#         if not np.isscalar(w_0):
#             w_0 = w_0.astype(float)

#     w = params.get('W_init', np.zeros_like(z, dtype=float))

#     # Construct S, St
#     if fix_zeros:
#         S, St = sum_squareform(n, edge_mask)
#     else:
#         S, St = sum_squareform(n)

#     K_op = lambda w_: S.dot(w_)
#     Kt_op = lambda z_: St.dot(z_)

#     if fix_zeros:
#         # Estimate norm_K
#         x_test = np.random.randn(S.shape[1])
#         for _ in range(5):
#             x_test = St.dot(S.dot(x_test))
#             norm_est = np.linalg.norm(x_test)
#             if norm_est == 0:
#                 break
#             x_test = x_test / norm_est
#         norm_K = sqrt(norm_est)
#     else:
#         norm_K = sqrt(2*(n-1))

#     def f_eval(w_):
#         return 2*np.dot(w_, z)
#     def f_prox(w_, c_):
#         return np.minimum(max_w, np.maximum(0, w_ - 2*c_*z))

#     def g_eval(x):
#         # Check if x has non-positive values
#         if np.any(x <= 0):
#             return np.inf
#         return -a * np.sum(np.log(x))

#     def g_prox(z_, c_):
#         sol, _ = prox_sum_log(z_, c_*a, param={'verbose':0})
#         return sol

#     def g_star_prox(z_, c_):
#         z_ = np.asarray(z_, dtype=float)
#         # If z_/(c_*a) not positive, prox_sum_log will report inf and handle gracefully
#         sol, _ = prox_sum_log(z_/(c_*a), 1/(c_*a), param={'verbose':0})
#         return z_ - c_*a * sol

#     if w_0 == 0:
#         def h_eval(w_):
#             return b*np.sum(w_**2)
#         def h_grad(w_):
#             return 2*b*w_
#         h_beta = 2*b
#     else:
#         def h_eval(w_):
#             return b*np.sum(w_**2) + c*np.sum((w_-w_0)**2)
#         def h_grad(w_):
#             return 2*((b+c)*w_ - c*w_0)
#         h_beta = 2*(b+c)

#     mu = h_beta + norm_K
#     epsilon = lin_map(0.0, [0, 1/(1+mu)], [0,1])
#     gn = lin_map(step_size, [epsilon, (1-epsilon)/mu], [0,1])

#     v_n = K_op(w)

#     stat = {}
#     if verbosity > 1:
#         stat['f_eval'] = np.full(maxit, np.nan)
#         stat['g_eval'] = np.full(maxit, np.nan)
#         stat['h_eval'] = np.full(maxit, np.nan)
#         stat['fgh_eval'] = np.full(maxit, np.nan)
#         stat['pos_violation'] = np.full(maxit, np.nan)
#         print('Relative change of primal, dual variables, and objective fun')

#     t0 = time.time()
#     for i in range(maxit):
#         Y_n = w - gn*(h_grad(w) + Kt_op(v_n))
#         y_n = v_n + gn*(K_op(w))

#         P_n = f_prox(Y_n, gn)
#         p_n = g_star_prox(y_n, gn)
#         Q_n = P_n - gn*(h_grad(P_n) + Kt_op(p_n))
#         q_n = p_n + gn*(K_op(P_n))

#         if verbosity > 2:
#             stat['f_eval'][i] = f_eval(w)
#             val_g = g_eval(K_op(w))
#             stat['g_eval'][i] = val_g
#             stat['h_eval'][i] = h_eval(w)
#             # If val_g is inf, fgh_eval might be inf as well, which is acceptable
#             stat['fgh_eval'][i] = stat['f_eval'][i] + stat['g_eval'][i] + stat['h_eval'][i]
#             stat['pos_violation'][i] = -np.sum(np.minimum(0,w))

#         denom_w = np.linalg.norm(w)
#         if denom_w < 1e-15:
#             denom_w = 1e-15
#         denom_v = np.linalg.norm(v_n)
#         if denom_v < 1e-15:
#             denom_v = 1e-15

#         rel_norm_primal = np.linalg.norm(-Y_n + Q_n)/denom_w
#         rel_norm_dual = np.linalg.norm(-y_n + q_n)/denom_v

#         if verbosity > 1 and verbosity <= 3:
#             print(f'iter {i+1:4d}: {rel_norm_primal:6.4e}   {rel_norm_dual:6.4e}')

#         w = w - Y_n + Q_n
#         v_n = v_n - y_n + q_n

#         if rel_norm_primal < tol and rel_norm_dual < tol:
#             break

#     stat['time'] = time.time() - t0
#     if verbosity > 0:
#         obj_val = f_eval(w) + g_eval(K_op(w)) + h_eval(w)
#         print(f'# iters: {i+1:4d}. Rel primal: {rel_norm_primal:6.4e} Rel dual: {rel_norm_dual:6.4e}  OBJ {obj_val:6.3e}')
#         print(f'Time needed is {stat["time"]} seconds')

#     if fix_zeros:
#         full_w = np.zeros(l, dtype=float)
#         full_w[ind] = w
#         w = full_w

#     if Z.ndim == 2 and Z.shape[0] == Z.shape[1]:
#         W = squareform_sp(w)
#     else:
#         W = w

#     return W, stat

In [11]:
def gsp_learn_graph_log_degrees(Z, a, b, params=None):
    if params is None:
        params = {}
    verbosity = params.get('verbosity', 1)
    maxit = params.get('maxit', 1000)
    tol = params.get('tol', 1e-5)
    step_size = params.get('step_size', 0.5)
    fix_zeros = params.get('fix_zeros', False)
    max_w = params.get('max_w', np.inf)

    w_0 = params.get('w_0', 0)
    if w_0 != 0 and 'c' not in params:
        raise ValueError("When params.w_0 is specified, c must also be specified")
    c = params.get('c', 0.0 if w_0 == 0 else None)

    # Convert Z to vector form
    if Z.ndim == 2 and Z.shape[0] == Z.shape[1]:
        z = squareform_sp(Z)
    else:
        z = Z
    # z = z.ravel()
    # z = z.toarray().ravel()
    if hasattr(z, "toarray"):  # For sparse matrices
        z = z.toarray().ravel()
    elif isinstance(z, np.ndarray):  # For dense numpy arrays
        z = z.ravel()
    else:
        raise TypeError("Input must be a sparse matrix or a numpy array.")
    
    l = len(z)
    n = int(round((1 + sqrt(1+8*l))/2))

    if not np.isscalar(w_0):
        # Convert w_0 to vector form if needed
        if w_0.ndim == 2 and w_0.shape[0] == w_0.shape[1]:
            w_0 = squareform_sp(w_0)
        w_0 = w_0.ravel()
    else:
        w_0 = float(w_0)

    if fix_zeros:
        edge_mask = params.get('edge_mask', None)
        if edge_mask is None:
            raise ValueError("edge_mask must be provided when fix_zeros is True")
        if edge_mask.ndim == 2 and edge_mask.shape[0] == edge_mask.shape[1]:
            edge_mask = squareform_sp(edge_mask)
        # edge_mask = edge_mask.ravel()
        edge_mask = edge_mask.toarray().ravel()
        ind = np.flatnonzero(edge_mask)
        z = z[ind].astype(float)
        if not np.isscalar(w_0):
            w_0 = w_0[ind].astype(float)
    else:
        z = z.astype(float)
        if not np.isscalar(w_0):
            w_0 = w_0.astype(float)

    w = params.get('W_init', np.zeros_like(z, dtype=float))

    # Construct S, St
    if fix_zeros:
        S, St = sum_squareform(n, edge_mask)
    else:
        S, St = sum_squareform(n)

    K_op = lambda w_: S.dot(w_)
    Kt_op = lambda z_: St.dot(z_)

    if fix_zeros:
        norm_K = normest(S)
    else:
        norm_K = sqrt(2*(n-1))

    def f_eval(w_):
        return 2*np.dot(w_, z)
    def f_prox(w_, c_):
        return np.minimum(max_w, np.maximum(0, w_ - 2*c_*z))

    def g_eval(x):
        # if x<=0, log not defined => infinite
        if np.any(x <= 0):
            return np.inf
        return -a * np.sum(np.log(x))
    def g_prox(z_, c_):
        sol, _ = prox_sum_log(z_, c_*a, param={'verbose':-3})
        return sol

    def g_star_prox(z_, c_):
        z_ = np.asarray(z_, dtype=float)
        sol, _ = prox_sum_log(z_/(c_*a), 1/(c_*a), param={'verbose':-3})
        return z_ - c_*a * sol

    if w_0 == 0:
        def h_eval(w_):
            return b*np.sum(w_**2)
        def h_grad(w_):
            return 2*b*w_
        h_beta = 2*b
    else:
        def h_eval(w_):
            return b*np.sum(w_**2) + c*np.sum((w_-w_0)**2)
        def h_grad(w_):
            return 2*((b+c)*w_ - c*w_0)
        h_beta = 2*(b+c)

    mu = h_beta + norm_K
    epsilon = lin_map(0.0, [0, 1/(1+mu)], [0,1])
    gn = lin_map(step_size, [epsilon, (1-epsilon)/mu], [0,1])

    v_n = K_op(w)

    stat = {}
    if verbosity > 1:
        stat['f_eval'] = np.full(maxit, np.nan)
        stat['g_eval'] = np.full(maxit, np.nan)
        stat['h_eval'] = np.full(maxit, np.nan)
        stat['fgh_eval'] = np.full(maxit, np.nan)
        stat['pos_violation'] = np.full(maxit, np.nan)
        if verbosity > 1:
            print('Relative change of primal, dual variables, and objective fun')

    t0 = time.time()
    for i in range(maxit):
        Y_n = w - gn*(h_grad(w) + Kt_op(v_n))
        y_n = v_n + gn*(K_op(w))

        P_n = f_prox(Y_n, gn)
        p_n = g_star_prox(y_n, gn) 
        Q_n = P_n - gn*(h_grad(P_n) + Kt_op(p_n))
        q_n = p_n + gn*(K_op(P_n))

        if verbosity > 2:
            stat['f_eval'][i] = f_eval(w)
            val_g = g_eval(K_op(w))
            stat['g_eval'][i] = val_g
            stat['h_eval'][i] = h_eval(w)
            stat['fgh_eval'][i] = stat['f_eval'][i] + stat['g_eval'][i] + stat['h_eval'][i]
            stat['pos_violation'][i] = -np.sum(np.minimum(0,w))

        denom_w = np.linalg.norm(w)
        if denom_w < 1e-15:
            denom_w = 1e-15
        denom_v = np.linalg.norm(v_n)
        if denom_v < 1e-15:
            denom_v = 1e-15

        rel_norm_primal = np.linalg.norm(-Y_n + Q_n)/denom_w
        rel_norm_dual = np.linalg.norm(-y_n + q_n)/denom_v

        if verbosity > 1 and verbosity <= 3:
            print(f'iter {i+1:4d}: {rel_norm_primal:6.4e}   {rel_norm_dual:6.4e}')

        w = w - Y_n + Q_n
        v_n = v_n - y_n + q_n

        if rel_norm_primal < tol and rel_norm_dual < tol:
            break

    stat['time'] = time.time() - t0
    if verbosity > 0:
        obj_val = f_eval(w) + g_eval(K_op(w)) + h_eval(w)
        print(f'# iters: {i+1:4d}. Rel primal: {rel_norm_primal:6.4e} Rel dual: {rel_norm_dual:6.4e}  OBJ {obj_val:6.3e}')
        print(f'Time needed is {stat["time"]} seconds')

    if fix_zeros:
        # reconstruct full w
        full_w = np.zeros(l, dtype=float)
        full_w[ind] = w
        w = full_w

    # Convert back to matrix if Z was a matrix
    if Z.ndim == 2 and Z.shape[0] == Z.shape[1]:
        W = squareform_sp(w)
    else:
        W = w

    return W, stat

In [63]:

# Suppose we have data matrix X (m x n), where m = number of nodes, n = dimension
# For demonstration, create a random dataset of m=1000 nodes in a 5-dimensional space
m = 1000
n = 5
X = np.random.randn(m, n).astype(np.float64)

# Desired number of neighbors per node for the final graph
k = 2

# Approximate Nearest Neighbors using pynndescent
# We want roughly 2*k neighbors (like MATLAB 2*k+1) and possibly some extra
# 'n_neighbors' is the number of neighbors NNDescent tries to find
n_neighbors = 2 * k + 1

# Build the NNDescent index
clock_flann = time.time()
# NNDescent expects data as np.array (m x n)
index = NNDescent(X, n_neighbors=n_neighbors, metric='euclidean', random_state=42)
# Query on the same data to get neighbors and distances
# This returns indices and distances arrays of shape (m, n_neighbors)
indx_arr, dist_arr = index.neighbor_graph
time_flann = time.time() - clock_flann
print(f"Time for pynndescent (FLANN equivalent): {time_flann:.3f} seconds")

# indx_arr: shape (m, n_neighbors), each row gives indices of neighbors
# dist_arr: shape (m, n_neighbors), each row gives distances
# The first neighbor returned by NNDescent is the point itself with distance 0
# We want to match MATLAB code structure:
# They got indx, indy, dist arrays that can form a sparse matrix Z_sp
# Let's form coo_matrix:
# We'll create arrays row_indices, col_indices, values for sparse matrix
row_indices = []
col_indices = []
values = []

# Iterate over each node to collect neighbor information
for i in range(m):
    # indx_arr[i,:], dist_arr[i,:]
    # The first neighbor is usually the node itself
    # Skip the self-neighbor (distance=0 to itself)
    neighbor_ids = indx_arr[i, 1:]   # skip the first which is self
    dists = dist_arr[i, 1:]
    row_indices.append(np.full(len(neighbor_ids), i, dtype=np.int32))
    col_indices.append(neighbor_ids)
    values.append(dists**2)  # as code squares distances

row_indices = np.concatenate(row_indices)
col_indices = np.concatenate(col_indices)
values = np.concatenate(values)

# Construct sparse distance matrix: Z_sp
# We have at most m*(n_neighbors-1) edges (slightly more because each node has that many neighbors)
Z_sp = coo_matrix((values, (row_indices, col_indices)), shape=(m, m))
Z_sp = Z_sp.tocsr()

# # Symmetrize the distance matrix as in MATLAB:
# Z_sp = gsp_symmetrize(Z_sp, 'full')

# Z_sp = squareform_sp(Z_sp)

Z_sp = gsp_ann_distanz(X, k=4, metric_in='euclidean')
Z_sp = gsp_symmetrize(Z_sp, 'full')

# Now we have an approximate nearest neighbor distance matrix Z_sp
# The next step: "Z_sorted = Z_sorted(:, 2:end).^2; first row is zero"
# In MATLAB, they have Z_sorted from nn_distanz.
# We must obtain Z_sorted from the indices and distances. But we already have dist_arr from NNDescent sorted:
# dist_arr[i,:] is sorted by pynndescent? By default, NNDescent does not guarantee sorted order by distance.
# If not sorted, we must sort them ourselves. Let's sort distances per row except the first (self).
Z_arr = []
for i in range(m):
    # Extract neighbors and distances except self
    dists = dist_arr[i, 1:]
    sorted_dists = np.sort(dists)  # ascending
    Z_arr.append(sorted_dists**2)
Z_sorted = np.vstack(Z_arr)  # shape (m, n_neighbors-1)

# Now we can compute theta:
# Z_is_sorted = True since we sorted them ourselves
Z_is_sorted = False
# Suppose geometric mean = 0 (like MATLAB code)
geom_mean = 1
theta= gsp_compute_graph_learning_theta(Z_sp, k, geom_mean, Z_is_sorted)
print(f"Computed theta: {theta}")

# Learn the graph using log degrees
W2, info_2 = gsp_learn_graph_log_degrees(theta * Z_sp, 1, 1)


Time for pynndescent (FLANN equivalent): 0.063 seconds


/Users/paul_reitz/opt/miniconda3/envs/apache_sedona/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


Computed theta: 0.0856266170058587
# iters:  297. Rel primal: 9.8320e-06 Rel dual: 2.7359e-06  OBJ 9.500e+00
Time needed is 0.010689020156860352 seconds


In [ ]:
W2

array([[ 0.00000000e+00,  8.96581428e-07,  8.96581428e-07,
         1.29473096e-01,  1.23381381e-01],
       [ 8.96581428e-07,  0.00000000e+00,  9.99999879e-01,
        -8.97947584e-07,  5.60852909e-07],
       [ 8.96581428e-07,  9.99999879e-01,  0.00000000e+00,
        -8.97947584e-07,  5.60852909e-07],
       [ 1.29473096e-01, -8.97947584e-07, -8.97947584e-07,
         0.00000000e+00,  1.30983579e-01],
       [ 1.23381381e-01,  5.60852909e-07,  5.60852909e-07,
         1.30983579e-01,  0.00000000e+00]])

In [27]:
print(f"W2 shape: {W2.shape}")

W2 shape: (5, 5)


In [25]:


# def glmm(y, iterations, classes, spread=0.1, regul=0.15, norm_par=1.5):
#     """
#     Python version of the MATLAB function glmm_matlab.

#     Parameters
#     ----------
#     y : ndarray (m x n)
#         Data matrix with m samples and n features.
#     iterations : int
#         Number of iterations.
#     classes : int
#         Number of classes (clusters).
#     spread : float, optional
#         Default 0.1
#     regul : float, optional
#         Default 0.15
#     norm_par : float, optional
#         Default 1.5

#     Returns
#     -------
#     L : ndarray (n x n x classes)
#         Graph Laplacians for each class.
#     gamma_hat : ndarray (m x classes)
#         Cluster posterior probabilities.
#     mu : ndarray (n x classes)
#         Cluster means.
#     log_likelihood : ndarray (iterations,)
#         Log-likelihood at each iteration.
#     """
#     y = np.asarray(y, dtype=np.float64)
#     # Helper constants
#     delta = 2
#     m, n = y.shape  # m samples, n features

#     # Initialize arrays
#     L = np.zeros((n, n, classes))
#     W = np.zeros((n, n, classes))
#     sigma = np.zeros((n-1, n-1, classes))
#     mu = np.zeros((n, classes))
#     gamma_hat = np.zeros((m, classes))
#     p = np.zeros(classes)
#     vecl = np.zeros((n, n, classes))
#     vall = np.zeros((n, n, classes))
#     yl = np.zeros((m, n-1, classes))

#     # Initialization
#     for class_idx in range(classes):
#         L[..., class_idx] = spread * np.eye(n) - (spread / n) * np.ones((n, n))
#         mu_curr = np.mean(y, axis=0) + np.random.randn(n) * np.std(y, axis=0)
#         mu_curr = mu_curr - np.mean(mu_curr)
#         mu[:, class_idx] = mu_curr
#         p[class_idx] = 1.0 / classes

#     log_likelihood = np.zeros(iterations)

#     # Main loop
#     for it in range(iterations):
#         # Expectation step
#         # putting everything in eigenvector space of dim-1
#         pall = np.zeros(m)

#         for class_idx in range(classes):
#             # Eigen decomposition
#             eigvals, eigvecs = np.linalg.eig(L[..., class_idx])
#             # In MATLAB, [V,D] = eig(L) returns V as eigenvectors and D diag eigenvalues.
#             # np.linalg.eig returns eigenvalues as a vector and eigenvectors as columns of eigvecs
#             # We should ensure sorting if needed, but we'll assume identical behavior as MATLAB.
#             # Let's store directly:
#             vecl[..., class_idx] = eigvecs
#             vall[..., class_idx] = np.diag(eigvals)

#             # sigma = inv(vall(2:n,2:n,class) + regul*I)
#             # We take the sub-block from index 1 to end (2:n in MATLAB means skipping the first eigenvalue)
#             sub_eigvals = eigvals[1:]
#             Sigma_inv = np.diag(sub_eigvals) + regul * np.eye(n-1)
#             # inverse
#             Sigma = np.linalg.inv(Sigma_inv)
#             # Make symmetric
#             Sigma = (Sigma + Sigma.T) / 2
#             sigma[..., class_idx] = Sigma

#             # yl = (y - mu(:,class)') * vecl(:,2:n,class)
#             # mu[:,class_idx] is shape (n,)
#             # y is (m,n), mu is (n,), we want (y - mu') => (m,n)
#             # vecl(:,2:n,class_idx) => vecl[:,: ,class_idx], take from index 1 to n-1 for columns
#             Y_centered = y - mu[:, class_idx]
#             YL = Y_centered @ vecl[:, 1:, class_idx]
#             yl[..., class_idx] = YL

#             # pall = pall + p(class) * mvnpdf(yl(:,:,class), zeros(1,n-1), sigma(:,:,class))
#             # We assume mvnpdf(yl, mean=0, cov=sigma) returns a vector of length m
#             mvn_val = mvnpdf(YL, np.zeros(n-1), Sigma)
#             pall += p[class_idx] * mvn_val

#         # Avoid division by zero
#         pall[pall == 0] = 0.1

#         # compute gamma_hat
#         for class_idx in range(classes):
#             mvn_val = mvnpdf(yl[..., class_idx], np.zeros(n-1), sigma[..., class_idx])
#             gamma_hat[:, class_idx] = (p[class_idx] * mvn_val) / pall

#         # log-likelihood
#         log_likelihood[it] = np.sum(np.log(pall))

#         # Maximization step
#         for class_idx in range(classes):
#             # mu(:,class) = (gamma_hat(:,class)' * y) / sum(gamma_hat(:,class))
#             wght = gamma_hat[:, class_idx]
#             mu[:, class_idx] = (wght @ y) / np.sum(wght)

#             # yc = sqrt(gamma_hat(:,class)) .* (y - mu(:,class)')
#             # sqrt(gamma_hat(:,class)) is shape (m,)
#             # (y - mu[:,class_idx]) is (m,n)
#             # elementwise: we can do broadcast:
#             yc = (y - mu[:, class_idx]) * np.sqrt(wght)[:, None]

#             # Z = gsp_distanz(yc).^2
#             Z = gsp_distanz(yc)
#             Z = Z**2

#             # theta = mean(Z(:))/norm_par
#             theta = np.mean(Z) / norm_par

#             # W_curr = delta*gsp_learn_graph_log_degrees(Z./theta, 1, 1)
#             W_curr, _ = gsp_learn_graph_log_degrees(Z / theta, 1, 1, params={})
#             W_curr = delta * W_curr

#             p[class_idx] = np.sum(wght) / m

#             # Compute L(:,:,class)
#             # L = diag(sum(W)) - W
#             W_sum = np.sum(W_curr, axis=1)
#             L[..., class_idx] = np.diag(W_sum) - W_curr

#             # W_curr(W_curr<1e-3)=0
#             W_curr[W_curr < 1e-3] = 0
#             W[..., class_idx] = W_curr

#     return L, gamma_hat, mu, log_likelihood


In [26]:
# def glmm(y, iterations, classes, spread=0.1, regul=0.15, norm_par=1.5):
#     """
#     Python version of glmm_matlab.

#     Parameters
#     ----------
#     y : ndarray (m x n)
#         Data matrix with m samples and n features.
#     iterations : int
#         Number of iterations.
#     classes : int
#         Number of classes (clusters).
#     spread : float, optional
#         Default 0.1
#     regul : float, optional
#         Default 0.15
#     norm_par : float, optional
#         Default 1.5

#     Returns
#     -------
#     L : ndarray (n x n x classes)
#         Graph Laplacians for each class.
#     gamma_hat : ndarray (m x classes)
#         Cluster posterior probabilities.
#     mu : ndarray (n x classes)
#         Cluster means.
#     log_likelihood : ndarray (iterations,)
#         Log-likelihood at each iteration.
#     """
#     y = np.asarray(y, dtype=np.float64)
#     n = y.shape[1]
#     m = y.shape[0]

#     L = np.zeros((n,n,classes))
#     W = np.zeros((n,n,classes))
#     sigma = np.zeros((n-1,n-1,classes))
#     mu = np.zeros((n, classes))
#     gamma_hat = np.zeros((m, classes))
#     p = np.zeros(classes)
#     vecl = np.zeros((n,n,classes))
#     vall = np.zeros((n,n,classes))
#     yl = np.zeros((m, n-1, classes))

#     # Initialization
#     # mu_curr = mean(y) + randn(1,n).*std(y), then shift by mean again
#     # p(class) = 1/classes
#     for class_idx in range(classes):
#         L[:,:,class_idx] = spread*np.eye(n) - (spread/n)*np.ones((n,n))
#         mu_curr = np.mean(y, axis=0) + np.random.randn(n)*np.std(y,axis=0)
#         mu_curr = mu_curr - np.mean(mu_curr)
#         mu[:,class_idx] = mu_curr
#         p[class_idx] = 1.0/classes

#     log_likelihood = np.zeros(iterations)

#     for it in range(iterations):
#         # E-step
#         pall = np.zeros(m, dtype=np.float64)
#         for class_idx in range(classes):
#             # eigen decomposition
#             eigvals, eigvecs = np.linalg.eig(L[:,:,class_idx])
#             # Store eigenvectors and diag eigenvalues
#             vecl[:,:,class_idx] = eigvecs
#             vall[:,:,class_idx] = np.diag(eigvals)

#             sub_eigvals = eigvals[1:]
#             Sigma_inv = np.diag(sub_eigvals) + regul*np.eye(n-1)
#             Sigma = np.linalg.inv(Sigma_inv)
#             Sigma = (Sigma+Sigma.T)/2
#             sigma[:,:,class_idx] = Sigma

#             Y_centered = y - mu[:,class_idx]
#             YL = Y_centered @ vecl[:,1:,class_idx]
#             yl[:,:,class_idx] = YL

#             # mvnpdf(yl(:,:,class), zeros(1,n-1), sigma)
#             mvn_val = mvnpdf(YL, np.zeros(n-1), Sigma)
#             pall += p[class_idx]*mvn_val

#         pall[pall==0] = 0.1

#         for class_idx in range(classes):
#             mvn_val = mvnpdf(yl[:,:,class_idx], np.zeros(n-1), sigma[:,:,class_idx])
#             gamma_hat[:,class_idx] = (p[class_idx]*mvn_val)/pall

#         log_likelihood[it] = np.sum(np.log(pall))

#         # M-step
#         for class_idx in range(classes):
#             wght = gamma_hat[:,class_idx]
#             mu[:,class_idx] = (wght @ y)/np.sum(wght)

#             yc = (y - mu[:,class_idx])*np.sqrt(wght)[:,None]
#             Z = gsp_distanz(yc)**2
#             theta = np.mean(Z)/norm_par

#             # W_curr = delta*gsp_learn_graph_log_degrees(Z./theta, 1,1)
#             # delta = 2 from code 
#             delta = 2
#             W_curr, _ = gsp_learn_graph_log_degrees(Z/theta, 1, 1, params={})
#             W_curr = delta*W_curr

#             p[class_idx] = np.sum(wght)/m
#             # L(:,:,class) = diag(sum(W)) - W
#             W_sum = np.sum(W_curr, axis=1)
#             L[:,:,class_idx] = np.diag(W_sum)-W_curr
#             W_curr[W_curr<1e-3] = 0
#             W[:,:,class_idx] = W_curr

#     return L, gamma_hat, mu, log_likelihood

In [64]:
def glmm(y, iterations, classes, avg_nr_edges , spread=0.1, regul=0.15, norm_par=1.5, alpha=None):
    """
    Python version of glmm_matlab with custom cluster priors (alpha).

    Parameters
    ----------
    y : ndarray (m x n)
        Data matrix with m samples and n features.
    iterations : int
        Number of iterations.
    classes : int
        Number of classes (clusters).
    spread : float, optional
        Default 0.1
    regul : float, optional
        Default 0.15
    norm_par : float, optional
        Default 1.5
    alpha : array-like, optional
        Mixing coefficients for the clusters. Should sum to 1 and have length = classes.
        If None, alpha is initialized to 1/classes each.

    Returns
    -------
    L : ndarray (n x n x classes)
        Graph Laplacians for each class.
    gamma_hat : ndarray (m x classes)
        Cluster posterior probabilities.
    mu : ndarray (n x classes)
        Cluster means.
    log_likelihood : ndarray (iterations,)
        Log-likelihood at each iteration.
    """

    y = np.asarray(y, dtype=np.float64)
    n = y.shape[1]
    m = y.shape[0]

    L = np.zeros((n,n,classes))
    W = np.zeros((n,n,classes))
    sigma = np.zeros((n-1,n-1,classes))
    mu = np.zeros((n, classes))
    gamma_hat = np.zeros((m, classes))
    p = np.zeros(classes)
    vecl = np.zeros((n,n,classes))
    vall = np.zeros((n,n,classes))
    yl = np.zeros((m, n-1, classes))

    # Initialization
    # If alpha is provided, use it. Otherwise, initialize uniformly.
    if alpha is not None:
        alpha = np.asarray(alpha, dtype=float)
        if len(alpha) != classes:
            raise ValueError("Length of alpha must match number of classes.")
        if not np.allclose(np.sum(alpha), 1.0):
            raise ValueError("Alpha must sum to 1.")
        p = alpha
    else:
        p[:] = 1.0/classes

    for class_idx in range(classes):
        L[:,:,class_idx] = spread*np.eye(n) - (spread/n)*np.ones((n,n))
        mu_curr = np.mean(y, axis=0) + np.random.randn(n)*np.std(y,axis=0)
        mu_curr = mu_curr - np.mean(mu_curr)
        mu[:,class_idx] = mu_curr

    log_likelihood = np.zeros(iterations)

    for it in range(iterations):
        # E-step
        pall = np.zeros(m, dtype=np.float64)
        for class_idx in range(classes):
            # eigen decomposition
            eigvals, eigvecs = np.linalg.eig(L[:,:,class_idx])
            vecl[:,:,class_idx] = eigvecs
            vall[:,:,class_idx] = np.diag(eigvals)

            sub_eigvals = eigvals[1:]
            Sigma_inv = np.diag(sub_eigvals) + regul*np.eye(n-1)
            Sigma = np.linalg.inv(Sigma_inv)
            Sigma = (Sigma+Sigma.T)/2
            sigma[:,:,class_idx] = Sigma

            Y_centered = y - mu[:,class_idx]
            YL = Y_centered @ vecl[:,1:,class_idx]
            yl[:,:,class_idx] = YL

            mvn_val = mvnpdf(YL, np.zeros(n-1), Sigma)
            pall += p[class_idx]*mvn_val

        pall[pall==0] = 0.1

        for class_idx in range(classes):
            mvn_val = mvnpdf(yl[:,:,class_idx], np.zeros(n-1), sigma[:,:,class_idx])
            gamma_hat[:,class_idx] = (p[class_idx]*mvn_val)/pall

        log_likelihood[it] = np.sum(np.log(pall))

        # M-step
        for class_idx in range(classes):
            wght = gamma_hat[:,class_idx]
            mu[:,class_idx] = (wght @ y)/np.sum(wght)

            yc = (y - mu[:,class_idx])*np.sqrt(wght)[:,None]
            # Z = gsp_distanz(yc)**2
            Z = gsp_ann_distanz(yc, k=avg_nr_edges, metric_in='euclidean')**2
            Z = gsp_symmetrize(Z, 'full')
            # theta = np.mean(Z)/norm_par
            theta = gsp_compute_graph_learning_theta(Z, avg_nr_edges)
            print(theta)

            # delta = 2 from code 
            delta = 2
            W_curr, _ = gsp_learn_graph_log_degrees(theta * Z, 1, 1, params={})
            W_curr = delta*W_curr

            p[class_idx] = np.sum(wght)/m
            W_sum = np.sum(W_curr, axis=1)
            L[:,:,class_idx] = np.diag(W_sum)-W_curr
            W_curr[W_curr<1e-3] = 0
            W[:,:,class_idx] = W_curr

    return L, gamma_hat, mu, log_likelihood




In [65]:



# Example parameters
n = 15  # graph size
m = 150 # number of signals
k = 2   # number of clusters
zero_thresh = 10e-4
p = np.linspace(0, 1, k+1)  # p = 0:1/k:1 in MATLAB
print(p)

# Generate graphs as MATLAB does
g = []
for i in range(k):
    g.append(generate_connected_graph(n, 0.7, zero_thresh, maxit=10, verbose=2))

# Generate gamma and gamma_cut
gamma = np.random.rand(m, 1)  # gamma = rand([m,1]) in MATLAB
gamma_cut = np.zeros((m, k))

dist = 0.5
y = np.zeros((m, n))
true_y = np.zeros((m, n, k))
center = np.zeros((n, k))
gauss = np.zeros((n, n, k))
Lap = np.zeros((n, n, k))

for i in range(k):
    # In MATLAB: gc = pinv(full(g(i).L));
    # We have g[i] as a dict, use g[i]['L']
    L_mat = g[i]['L']
    gc = np.linalg.pinv(L_mat)
    gauss[:, :, i] = (gc + gc.T)/2
    Lap[:, :, i] = L_mat

    c = dist * np.random.randn(n)
    c = c - np.mean(c)
    center[:, i] = c

    # gamma_cut(p(i)<gamma & gamma<=p(i+1), i) = 1;
    mask = (gamma[:, 0] > p[i]) & (gamma[:, 0] <= p[i+1])
    gamma_cut[mask, i] = 1

    samples = np.random.multivariate_normal(center[:, i], gauss[:, :, i], m)
    samples = gamma_cut[:, i][:, np.newaxis] * samples
    true_y[:, :, i] = samples
    y += samples

# Now we train glmm on data y
# Assuming glmm function is defined and returns (Ls, gamma_hats, mus, log_likelihood)
avg_nr_edges = 5
iterations = 200
Ls, gamma_hats, mus, log_likelihood = glmm(y, iterations, k, avg_nr_edges, spread=0.1, regul=0.15, norm_par=1.5, alpha=None)
print('Training done')

print("sum(gamma_hats,1):", np.sum(gamma_hats, axis=0))

# If identify_and_compare returns (identify, precision, recall, f, cl_errors)
identify, precision, recall, f, cl_errors ,  NMI_scores, num_of_edges_arr = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)

print("Identify:", identify)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f)
print("Cluster Errors:", cl_errors)
print("NMI Scores:", NMI_scores)
print("Number of edges:", num_of_edges_arr)

summed_gamma_hats = np.sum(gamma_hats, axis=1)
are_all_elements_one = np.allclose(summed_gamma_hats, 1.0, atol=1e-8)
print("\nAre all elements in each row of gamma_hats summing to 1:", are_all_elements_one)


[0.  0.5 1. ]
A connected graph has been created in 1 iteration(s)
A connected graph has been created in 1 iteration(s)


/Users/paul_reitz/opt/miniconda3/envs/apache_sedona/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


10000000.0
# iters:   54. Rel primal: 8.4561e-06 Rel dual: 9.5588e-06  OBJ 2.330e+03
Time needed is 0.0023441314697265625 seconds


/Users/paul_reitz/opt/miniconda3/envs/apache_sedona/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


10000000.0
# iters:   54. Rel primal: 8.4442e-06 Rel dual: 9.5328e-06  OBJ 5.973e+03
Time needed is 0.002168893814086914 seconds


...

7972173.840000483
# iters:   48. Rel primal: 3.9285e-06 Rel dual: 9.4477e-06  OBJ 3.517e+03
Time needed is 0.0019750595092773438 seconds


/Users/paul_reitz/opt/miniconda3/envs/apache_sedona/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


8666666.675413875
# iters:   48. Rel primal: 3.9811e-06 Rel dual: 7.7601e-06  OBJ 1.895e+03
Time needed is 0.0017421245574951172 seconds


/Users/paul_reitz/opt/miniconda3/envs/apache_sedona/lib/python3.12/site-packages/pynndescent/pynndescent_.py:939: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


7972173.840017851
# iters:   48. Rel primal: 4.1794e-06 Rel dual: 9.1779e-06  OBJ 3.392e+03
Time needed is 0.0017049312591552734 seconds
Training done
sum(gamma_hats,1): [74.11097371 75.88902629]
Identify: [1 0]
Precision: [[0.71428571]
 [0.44067797]]
Recall: [[0.5625    ]
 [0.38235294]]
F-measure: [[0.62937063]
 [0.40944882]]
Cluster Errors: [0.00803498 0.00803498]
NMI Scores: [0.01585769 0.20078225]
Number of edges: [63. 59.]

Are all elements in each row of gamma_hats summing to 1: True


In [11]:
print("Identify:", identify)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f)
print("Cluster Errors:", cl_errors)
print("NMI Scores:", NMI_scores)
print("Number of edges:", num_of_edges_arr)

summed_gamma_hats = np.sum(gamma_hats, axis=1)
are_all_elements_one = np.allclose(summed_gamma_hats, 1.0, atol=1e-8)
print("\nAre all elements in each row of gamma_hats summing to 1:", are_all_elements_one)

NameError: name 'identify' is not defined

In [ ]:


# Parameters
n = 15  # graph size
m = 150 # number of signals
k = 2   # number of clusters
zero_thresh = 1e-4  

g = []
for i in range(k):
    g.append(generate_connected_graph(n, 0.7, zero_thresh, maxit=10, verbose=1))

gamma = np.random.rand(m, 1) 
gamma_cut = np.zeros((m, k))
dist = 0.5
p = np.linspace(0, 1, k+1)
y = np.zeros((m, n))
true_y = np.zeros((m, n, k))
center = np.zeros((n, k))
gauss = np.zeros((n, n, k))
Lap = np.zeros((n, n, k))

for i in range(k):
    L_mat = g[i]['L']
    gc = np.linalg.pinv(L_mat)
    gauss[:, :, i] = (gc + gc.T) / 2
    Lap[:, :, i] = L_mat
    c = dist * np.random.randn(n)
    c = c - np.mean(c)
    center[:, i] = c
    mask = (gamma[:, 0] > p[i]) & (gamma[:, 0] <= p[i+1])
    gamma_cut[mask, i] = 1
    samples = np.random.multivariate_normal(center[:, i], gauss[:, :, i], m)
    # Multiply each row by gamma_cut[:,i]
    samples = gamma_cut[:, i][:, np.newaxis] * samples
    true_y[:, :, i] = samples
    y += samples

# Now train glmm on data y
iterations = 200
Ls, gamma_hats, mus, log_likelihood = glmm(y, iterations, k, spread=0.1, regul=0.15, norm_par=1.5, alpha=None)
print('Training done')
print("sum(gamma_hats,1):", np.sum(gamma_hats, axis=0))

identify, precision, recall, f, cl_errors, NMI_scores, num_of_edges_arr = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)

print("Identify:", identify)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f)
print("Cluster Errors:", cl_errors)
print("NMI Scores:", NMI_scores)
print("Number of edges:", num_of_edges_arr)

summed_gamma_hats = np.sum(gamma_hats, axis=1)
are_all_elements_one = np.allclose(summed_gamma_hats, 1.0, atol=1e-8)
print("\nAre all elements in each row of gamma_hats summing to 1:", are_all_elements_one)


TypeError: glmm() missing 1 required positional argument: 'avg_nr_edges'

In [ ]:


# Set parameters according to the theoretical setup:
n = 15   # graph size
m = 150  # number of signals
k = 2    # number of clusters
zero_thresh = 1e-4

# Generate k connected Erdos-Renyi graphs as in the MATLAB code
g = []
for i in range(k):
    g.append(generate_connected_graph(n, 0.7, zero_thresh, maxit=10, verbose=1))

# Generate cluster memberships and signals according to the setup:
# gamma = rand(m,1) ~ uniform random in [0,1)
# p = [0, 1/k, 2/k, ..., 1] to define cluster assignments
gamma = np.random.rand(m, 1)
gamma_cut = np.zeros((m, k))
dist = 0.5
p = np.linspace(0, 1, k+1)  # for k=2, p=[0,0.5,1]; for balanced clusters {0.5,0.5}

y = np.zeros((m, n))
true_y = np.zeros((m, n, k))
center = np.zeros((n, k))
gauss = np.zeros((n, n, k))
Lap = np.zeros((n, n, k))

# According to the theory, each cluster k has:
#  - A mean vector µ_k ~ N(0, σ^2 I) with σ=0.5, done by dist * randn and then centering.
#  - Signals from cluster k are: x_m ∼ N(µ_k, L_k^\dagger)
# The code:
for i in range(k):
    L_mat = g[i]['L']          # Retrieve the Laplacian from the generated graph
    gc = np.linalg.pinv(L_mat) # Pseudoinverse for covariance
    gauss[:, :, i] = (gc + gc.T) / 2
    Lap[:, :, i] = L_mat

    # Generate center vector µ_k as per σ=0.5
    c = dist * np.random.randn(n)
    c = c - np.mean(c)  # ensure mean is zero
    center[:, i] = c

    # Assign signals to cluster i according to p:
    mask = (gamma[:, 0] > p[i]) & (gamma[:, 0] <= p[i+1])
    gamma_cut[mask, i] = 1

    # Generate signals for cluster i
    # x_m ∼ N(µ_i, gauss[:,:,i]) if in cluster i, else 0
    samples = np.random.multivariate_normal(center[:, i], gauss[:, :, i], m)
    samples = gamma_cut[:, i][:, np.newaxis] * samples  # 0 out for non-members
    true_y[:, :, i] = samples
    y += samples

# Now we train the GLMM on data y:
# Using glmm as previously defined
iterations = 200
y= normalize_data(y)
Ls, gamma_hats, mus, log_likelihood = glmm(y, iterations, k, spread=0.1, regul=0.15, norm_par=1.5, alpha=None)
print('Training done')

# Check sum(gamma_hats,1) as in MATLAB (sum along rows in MATLAB is axis=0 in Python)
print("sum(gamma_hats,1):", np.sum(gamma_hats, axis=0))

# Identify and compare results (assuming identify_and_compare returns identify, precision, recall, f, cl_errors, NMI_scores, num_of_edges_arr)
identify, precision, recall, f, cl_errors, NMI_scores, num_of_edges_arr = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)

print("Identify:", identify)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f)
print("Cluster Errors:", cl_errors)
print("NMI Scores:", NMI_scores)
print("Number of edges:", num_of_edges_arr)

# Check if rows of gamma_hats sum to 1
summed_gamma_hats = np.sum(gamma_hats, axis=1)
are_all_elements_one = np.allclose(summed_gamma_hats, 1.0, atol=1e-8)
print("\nAre all elements in each row of gamma_hats summing to 1:", are_all_elements_one)


# iters:   97. Rel primal: 9.5838e-06 Rel dual: 3.4506e-06  OBJ 1.500e+01
Time needed is 0.02560281753540039 seconds
# iters:   98. Rel primal: 9.5655e-06 Rel dual: 2.7603e-06  OBJ 1.415e+01
Time needed is 0.008249044418334961 seconds
# iters:  106. Rel primal: 9.1393e-06 Rel dual: 1.9620e-06  OBJ 1.537e+01
Time needed is 0.0055599212646484375 seconds
# iters:   92. Rel primal: 9.7356e-06 Rel dual: 9.8120e-06  OBJ 1.437e+01
Time needed is 0.004853010177612305 seconds
# iters:  107. Rel primal: 9.0082e-06 Rel dual: 2.7974e-06  OBJ 1.575e+01
Time needed is 0.0041658878326416016 seconds
# iters:  110. Rel primal: 9.1817e-06 Rel dual: 1.5878e-06  OBJ 1.508e+01
Time needed is 0.008427858352661133 seconds
# iters:  116. Rel primal: 9.5126e-06 Rel dual: 1.7685e-06  OBJ 1.616e+01
Time needed is 0.0052149295806884766 seconds
# iters:  112. Rel primal: 9.9245e-06 Rel dual: 2.2498e-06  OBJ 1.570e+01
Time needed is 0.004464149475097656 seconds
# iters:  120. Rel primal: 9.8291e-06 Rel dual: 2.1313

In [ ]:


# Set parameters according to the theoretical setup:
n = 15   # graph size
m = 150  # number of signals
k = 2    # number of clusters
zero_thresh = 1e-4

# Generate k connected Erdos-Renyi graphs as in the MATLAB code
g = []
for i in range(k):
    g.append(generate_connected_graph(n, 0.7, zero_thresh, maxit=10, verbose=1))

# Generate cluster memberships and signals according to the setup:
# gamma = rand(m,1) ~ uniform random in [0,1)
# p = [0, 1/k, 2/k, ..., 1] to define cluster assignments
gamma = np.random.rand(m, 1)
gamma_cut = np.zeros((m, k))
dist = 0.5
p = np.linspace(0, 1, k+1)  # for k=2, p=[0,0.5,1]; for balanced clusters {0.5,0.5}

y = np.zeros((m, n))
true_y = np.zeros((m, n, k))
center = np.zeros((n, k))
gauss = np.zeros((n, n, k))
Lap = np.zeros((n, n, k))

# According to the theory, each cluster k has:
#  - A mean vector µ_k ~ N(0, σ^2 I) with σ=0.5, done by dist * randn and then centering.
#  - Signals from cluster k are: x_m ∼ N(µ_k, L_k^\dagger)
# The code:
for i in range(k):
    L_mat = g[i]['L']          # Retrieve the Laplacian from the generated graph
    gc = np.linalg.pinv(L_mat) # Pseudoinverse for covariance
    gauss[:, :, i] = (gc + gc.T) / 2
    Lap[:, :, i] = L_mat

    # Generate center vector µ_k as per σ=0.5
    c = dist * np.random.randn(n)
    c = c - np.mean(c)  # ensure mean is zero
    center[:, i] = c

    # Assign signals to cluster i according to p:
    mask = (gamma[:, 0] > p[i]) & (gamma[:, 0] <= p[i+1])
    gamma_cut[mask, i] = 1

    # Generate signals for cluster i
    # x_m ∼ N(µ_i, gauss[:,:,i]) if in cluster i, else 0
    samples = np.random.multivariate_normal(center[:, i], gauss[:, :, i], m)
    samples = gamma_cut[:, i][:, np.newaxis] * samples  # 0 out for non-members
    true_y[:, :, i] = samples
    y += samples

# Now we train the GLMM on data y:
# Using glmm as previously defined
iterations = 200
y= normalize_data(y)
in_alpha = [0.2,0.8]
Ls, gamma_hats, mus, log_likelihood = glmm(y, iterations, k, spread=0.1, regul=0.15, norm_par=1.5, alpha=in_alpha)
print('Training done')

# Check sum(gamma_hats,1) as in MATLAB (sum along rows in MATLAB is axis=0 in Python)
print("sum(gamma_hats,1):", np.sum(gamma_hats, axis=0))

# Identify and compare results (assuming identify_and_compare returns identify, precision, recall, f, cl_errors, NMI_scores, num_of_edges_arr)
identify, precision, recall, f, cl_errors, NMI_scores, num_of_edges_arr = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)

print("Identify:", identify)
print("Precision:", precision)
print("Recall:", recall)
print("F-measure:", f)
print("Cluster Errors:", cl_errors)
print("NMI Scores:", NMI_scores)
print("Number of edges:", num_of_edges_arr)

# Check if rows of gamma_hats sum to 1
summed_gamma_hats = np.sum(gamma_hats, axis=1)
are_all_elements_one = np.allclose(summed_gamma_hats, 1.0, atol=1e-8)
print("\nAre all elements in each row of gamma_hats summing to 1:", are_all_elements_one)


# iters:  110. Rel primal: 9.1618e-06 Rel dual: 2.0355e-06  OBJ 1.613e+01
Time needed is 0.013456106185913086 seconds
# iters:  102. Rel primal: 8.1768e-06 Rel dual: 4.6602e-06  OBJ 1.571e+01
Time needed is 0.01656508445739746 seconds
# iters:  124. Rel primal: 9.1600e-06 Rel dual: 8.2541e-07  OBJ 1.579e+01
Time needed is 0.02474498748779297 seconds
# iters:  100. Rel primal: 9.0307e-06 Rel dual: 3.7564e-06  OBJ 1.540e+01
Time needed is 0.003944873809814453 seconds
# iters:  121. Rel primal: 9.3033e-06 Rel dual: 1.4007e-06  OBJ 1.539e+01
Time needed is 0.018718957901000977 seconds
# iters:  101. Rel primal: 9.6315e-06 Rel dual: 2.9482e-06  OBJ 1.505e+01
Time needed is 0.015413999557495117 seconds
# iters:  120. Rel primal: 9.8210e-06 Rel dual: 1.6654e-06  OBJ 1.545e+01
Time needed is 0.008890151977539062 seconds
# iters:  101. Rel primal: 8.9978e-06 Rel dual: 2.0842e-06  OBJ 1.471e+01
Time needed is 0.010299921035766602 seconds
# iters:  119. Rel primal: 9.5956e-06 Rel dual: 1.5861e-06

In [ ]:

# n = 15  
# m = 150  
# k = 2
# zero_thresh = 10e-4

# g = [generate_connected_graph(n, 0.7, zero_thresh) for _ in range(k)]

# gamma = np.random.rand(m, 1)
# gamma_cut = np.zeros((m, k))
# dist = 0.5
# p = np.linspace(0, 1, k + 1)x
# y = np.zeros((m, n))
# true_y = np.zeros((m, n, k))
# center = np.zeros((n, k))
# gauss = np.zeros((n, n, k))
# Lap = np.zeros((n, n, k))

# for i in range(k):
#     gc = pinv(g[i])
#     gauss[:, :, i] = (gc + gc.T) / 2
#     Lap[:, :, i] = g[i]
#     center[:, i] = dist * np.random.randn(n)
#     center[:, i] = center[:, i] - np.mean(center[:, i])
#     gamma_cut[(p[i] < gamma[:, 0]) & (gamma[:, 0] <= p[i + 1]), i] = 1
#     true_y[:, :, i] = gamma_cut[:, i][:, np.newaxis] * np.random.multivariate_normal(center[:, i], gauss[:, :, i], m)
#     y += true_y[:, :, i]

# iterations = 200
# Ls, gamma_hats, mus, log_likelihood = glmm(y, iterations, k)
# print('Training done')
# # identify, precision, recall, f, cl_errors = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)
# identify, precision, recall, f, cl_errors, NMI_score, num_of_edges = identify_and_compare(Ls, Lap, gamma_hats, gamma_cut, k)

# print("Identify:", identify)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F-measure:", f)
# print("Cluster Errors:", cl_errors)
# print('Normalized mutual information', NMI_score)
# print("Number of estimated edges", num_of_edges)
# summed_gamma_hats = np.sum(gamma_hats, axis=1)
# summed_gamma_hats_column = summed_gamma_hats[:, np.newaxis]
# are_all_elements_one = np.allclose(summed_gamma_hats_column, 1, atol=1e-8)
# print("\nAre all elements in the colum wise summed gamma_hat equal to 1:", are_all_elements_one)
# visualize_glmm(Ls, gamma_hats)